In [1]:
from keras.layers import Input, concatenate, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization
from keras.layers.core import Dropout
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from glob import glob
import re
import os.path
import scipy.misc
import imageio
from skimage.transform import resize
from skimage.measure import moments_hu
import numpy as np
import random
import matplotlib.pyplot as plt
from keras import optimizers
from keras.callbacks import ModelCheckpoint
import keras.backend as K
namefile = 'Unet_Cabeza_Mano'
image_shape = (160, 320)  
data_dir = './manos'
batch_size = 5

Using TensorFlow backend.


In [2]:
def generator(data_folder, image_shape, batch_size):
    """  Create batches of training and validation data """
    image_paths = glob(os.path.join(data_folder, 'images', '*.png'))
    label_paths = {
        re.sub(r'_(lane|road)_', '_', os.path.basename(path)): path
        for path in glob(os.path.join(data_folder, 'labels', '*.png'))}
    
    background_color = np.array([0.9, 0.9, 0.9])  ## Solo el blanco

    random.shuffle(image_paths)
    while 1: 
        for batch_i in range(0, len(image_paths), batch_size):
            images = []
            gt_images = []
            for image_file in image_paths[batch_i:batch_i+batch_size]:
                gt_image_file = label_paths[os.path.basename(image_file)]

                image = resize(imageio.imread(image_file), image_shape, mode='constant',anti_aliasing=False)
                gt_image = resize(imageio.imread(gt_image_file), image_shape, mode='constant', anti_aliasing=False)

                
                gt_bg = np.all(gt_image >= background_color, axis=2)  #Binarizar la segmentación del target
                gt_bg = gt_bg.reshape(*gt_bg.shape, 1)   # Añadirle una dimensión al tensor
                gt_image = np.concatenate((gt_bg, np.invert(gt_bg)), axis=2)  #crear dos capas reflejo
                
                images.append(image)
                gt_images.append(gt_image)
                
            X = np.array(images)
            Y = np.array(gt_images)
            yield (X, Y)
        
train_generator = generator(os.path.join(data_dir, 'training'), image_shape, batch_size)
validation_generator = generator(os.path.join(data_dir, 'validation'), image_shape, batch_size)      


In [ ]:
image_file ='./manos/training/images/LSM2740_65.png'
gt_image_file ='./manos/training/labels/LSM2740_65.png'

image = resize(imageio.imread(image_file), image_shape, mode='constant',anti_aliasing=False)
gt_image = resize(imageio.imread(gt_image_file), image_shape, mode='constant', anti_aliasing=False)

In [ ]:
print(gt_image.shape)
background_color = np.array([0.9, 0.9, 0.9])  ## Solo el blanco
gt_bg = np.all(gt_image >= background_color, axis=2)
print(gt_bg.shape)
gt_bg = gt_bg.reshape(*gt_bg.shape, 1)
print(gt_bg.shape)
gt_image = np.concatenate((gt_bg, np.invert(gt_bg)), axis=2)
print(gt_image.shape)

In [2]:
data_gen_args = dict(featurewise_center=True
                     )
image_datagen = ImageDataGenerator(**data_gen_args)
mask_datagen = ImageDataGenerator(**data_gen_args)


In [3]:
seed = 1
image_generator = image_datagen.flow_from_directory(
    'manos1/training/images',
     target_size=image_shape,
     class_mode='input',
     seed=seed)

mask_generator = mask_datagen.flow_from_directory(
    'manos1/training/labels',
    target_size=image_shape,
    class_mode='input',
    seed=seed)



Found 29 images belonging to 2 classes.
Found 29 images belonging to 2 classes.


In [ ]:
image_generator.batch_size

In [4]:
train_generator = zip(image_generator, mask_generator)


In [3]:
#Función de validación
def IOU_calc(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    union = K.sum(K.clip(y_true_f + y_pred_f, 0, 1))
    return intersection/union


def IOU_calc_loss(y_true, y_pred):
    return -IOU_calc(y_true, y_pred)

In [2]:
#Modelo de Unet 
def unet02(img_rows, img_cols, img_channels):  #lr = 0.0009
    x = Input(shape=(img_rows, img_cols, img_channels))
    
    # Encoder 
    conv1 = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    conv2 = Conv2D(64, (1, 1), padding='same', activation='relu')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv2)
    
    conv3 = Conv2D(128, (3, 3), padding='same', activation='relu')(pool1)
    conv4 = Conv2D(128, (1, 1), padding='same', activation='relu')(conv3)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv4)
    
    conv5 = Conv2D(256, (3, 3), padding='same', activation='relu')(pool2)
    conv6 = Conv2D(256, (1, 1), padding='same', activation='relu')(conv5)
    
    # Decoder   
    convT1 = Conv2D(128, (2, 2), padding='same', activation='relu')(UpSampling2D(size=(2, 2))(conv6))
    merge1 = concatenate([convT1, conv4], axis=3)
    conv7 = Conv2D(128, (3, 3), padding='same', activation='relu')(merge1)
    conv8 = Conv2D(128, (1, 1), padding='same', activation='relu')(conv7)
    
    convT2 = Conv2D(64, (2, 2), padding='same', activation='relu')(UpSampling2D(size=(2, 2))(conv8))
    merge2 = concatenate([convT2, conv2], axis=3)
    conv9 = Conv2D(64, (3, 3), padding='same', activation='relu')(merge2)
    conv10 = Conv2D(64, (1, 1), padding='same', activation='relu')(conv9)  

    # Segmentation
    y = Conv2D(2, (1, 1), activation='softmax')(conv10)

    return Model(inputs=x, outputs=y)

In [5]:
model = unet02(image_shape[0],image_shape[1],3)

ValueError: A `Concatenate` layer requires inputs with matching shapes except for the concat axis. Got inputs shapes: [(None, 100, 124, 128), (None, 100, 125, 128)]

In [ ]:
#fase de entrenamineto

model = unet02(image_shape[0],image_shape[1],3)
adam = optimizers.Adam(lr=0.00009, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=[IOU_calc])
checkpointer = ModelCheckpoint(filepath='bestmodel_' + namefile + '.h5', monitor='val_loss', verbose=0, save_best_only=True,
                              mode='min', period=1)
h = model.fit_generator(train_generator, steps_per_epoch=200, epochs=2, verbose=1, callbacks=[checkpointer])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/2
150/200 [=====================>........] - ETA: 27s - loss: 0.0994 - IOU_calc: 0.8925

In [ ]:
#Graficas del entrenamiento

plt.rcParams['figure.figsize'] = (5,4) # Hacer las figuras más grandes
epoch_max = np.argmax(h.history['val_IOU_calc'])
plt.plot(h.history['val_IOU_calc'], label='IOUval')
plt.plot(h.history['IOU_calc'], label='IOUtrain')
plt.legend(loc='lower right')
plt.plot(epoch_max, h.history['val_IOU_calc'][epoch_max],'*')
plt.xlabel('epochs', fontsize=24)
plt.ylabel('IOU', fontsize=24)
plt.suptitle('Curvas de aprendizaje', fontsize=30)
plt.savefig('IOU_bestmodel_' + namefile + '.pdf', bbox_inches='tight')
plt.show()


epoch_max = np.argmin(h.history['val_loss'])
plt.plot(h.history['val_loss'], label='CEval')
plt.plot(h.history['loss'], label='CE')
plt.legend(loc='upper right')
plt.plot(epoch_max, h.history['val_loss'][epoch_max],'*')
plt.xlabel('epochs', fontsize=24)
plt.ylabel('cross entropy', fontsize=24)
plt.suptitle('Curvas de aprendizaje', fontsize=30)
plt.savefig('CE_bestmodel_' + namefile + '.pdf', bbox_inches='tight')
plt.show()


In [ ]:
# Evaluar el mejor modelo
model.load_weights('bestmodel_' + namefile + '.h5')

score_train = model.evaluate_generator(train_generator, steps=89//batch_size)
print('training', score_train)
score_val = model.evaluate_generator(validation_generator, steps=89//batch_size)
print('validation', score_val)

In [ ]:
#Muestra algunos resultados de la fase de entrenamiento

plt.rcParams['figure.figsize'] = (15,15) # Hacer las figuras más grandes
data_folder = './manos/validation/images'
th_segment = 0.5
k = 10
filelist = glob(os.path.join(data_folder, '*.png'))
random.shuffle(filelist)
i = 1
for image_file in filelist[0:k]:
    img_original = imageio.imread(image_file)
    img = resize(img_original, image_shape, mode = 'constant')
    y = model.predict(np.expand_dims(img, axis=0), batch_size=1)
    #print (y.shape)
    mask = np.stack([y[0,:,:,0] > th_segment,np.zeros(image_shape),np.zeros(image_shape)], axis=2)
    masked = np.ma.masked_array(mask, img).astype('float32')
    #print (masked.shape)
    #print (masked.dtype)
    plt.subplot(5,2,i)
    plt.imshow(img,interpolation='none') 
    plt.imshow(masked, interpolation='none', alpha=0.35) 
    plt.tick_params(axis='both',which='both',bottom=False, left=False,labelbottom=False,labelleft=False) 
    i += 1
#plt.savefig('photos_bestmodel_2' + namefile +'.pdf', bbox_inches='tight')
plt.show()

In [ ]:
#Cargar un Modelo
from keras.models import load_model
model = load_model('bestmodel_' + namefile + '.h5', custom_objects={'IOU_calc': IOU_calc}) # se carga el modelo completo

In [ ]:
#Cargar una imagen y realizar procesamiento para la entrada del Modelo
image_file1 = './manos/validation/images/S1.jpg'
img_original = imageio.imread(image_file1)
img = resize(img_original, image_shape ,mode = 'constant')

img1 = np.expand_dims(img, axis=0)
img1.shape

In [ ]:
#Se cre una función para ver ciertas capas de pendiendo de una entrada
from keras import backend as K

# with a Sequential model
get_3rd_layer_output = K.function([model.layers[0].input],
                                  [model.layers[6].output])

layer_output = get_3rd_layer_output([img1]) # img1 es la imagen de entrada, se introduce al modelo
ji = layer_output[0] #se optiene la lista de salida de la capa

print(ji.shape[3])

#plt.matshow(ji[0,:,:,12])

In [ ]:
#Se muestran  filtros dentro de la capa seleccionada
plt.rcParams['figure.figsize'] = (15,15) # Hacer las figuras más grandes
arr = np.arange(ji.shape[3])
np.random.shuffle(arr)
g=1
for i in arr[0:10]:
    
    plt.subplot(5,2,g)
    plt.imshow(ji[0,:,:,i],cmap='gray') 
   
    plt.title('número de filtro'+' '+ str(i), fontsize=20)
    plt.tick_params(axis='both',which='both',bottom=False, left=False,labelbottom=False,labelleft=False) 
    g += 1
plt.savefig('fotos_filtros' + namefile +'.pdf', bbox_inches='tight')
plt.show()

In [ ]:
th_segment = 0.5
y1 = model.predict(img1, batch_size=1)
#print (y.shape)
mask1 = np.stack([y1[0,:,:,0] > th_segment,np.zeros(image_shape),np.zeros(image_shape)], axis=2)
masked1 = np.ma.masked_array(mask1, img).astype('float32')
plt.imshow(img,interpolation='none') 
plt.imshow(masked1, interpolation='none', alpha=0.55) 